# Power Outage Durations

**Name(s)**: Aki Baidya and Dhruv Sehgal

**Website Link**: https://aki-baidya.github.io/power-outages/

In [77]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [78]:
pd.set_option('display.max_columns', None) # to see all columns

# loading in the dataset
# we first changed the extension from .xlsx.xls to .xlsx
try:
    unclean_df = pd.read_excel('outage.xlsx', header=5) # omitting the rows above since they are not part of the data
    unclean_df = unclean_df.drop(0) # to remove the row that talks about the format of some data columns
    unclean_df = unclean_df.drop(columns=unclean_df.columns[:2]) # removing the 'variables' and 'OBS' columns

except FileNotFoundError:
    print('Error: \'outage.xlsx\' not found.')
display(unclean_df.head())

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,NaN,NaN,3060,NaN,70000.0,11.6,9.18,6.81,9.28,2332915,2114774,2113291,6562520,35.55,32.23,32.2,2.31e+06,276286.0,10673.0,2.60e+06,88.94,10.64,0.41,51268,47586,1.08,1.6,4802,274182,1.75,2.2,5.35e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
2,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,vandalism,NaN,1,NaN,NaN,12.12,9.71,6.49,9.28,1586986,1807756,1887927,5284231,30.03,34.21,35.73,2.35e+06,284978.0,9898.0,2.64e+06,88.83,10.79,0.37,53499,49091,1.09,1.9,5226,291955,1.79,2.2,5.46e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
3,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,heavy wind,NaN,3000,NaN,70000.0,10.87,8.19,6.07,8.15,1467293,1801683,1951295,5222116,28.1,34.5,37.37,2.30e+06,276463.0,10150.0,2.59e+06,88.92,10.69,0.39,50447,47287,1.07,2.7,4571,267895,1.71,2.1,5.31e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
4,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,thunderstorm,NaN,2550,NaN,68200.0,11.79,9.25,6.71,9.19,1851519,1941174,1993026,5787064,31.99,33.54,34.44,2.32e+06,278466.0,11010.0,2.61e+06,88.9,10.68,0.42,51598,48156,1.07,0.6,5364,277627,1.93,2.2,5.38e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48
5,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,NaN,NaN,1740,250,250000.0,13.07,10.16,7.74,10.43,2028875,2161612,1777937,5970339,33.98,36.21,29.78,2.37e+06,289044.0,9812.0,2.67e+06,88.82,10.81,0.37,54431,49844,1.09,1.7,4873,292023,1.67,2.2,5.49e+06,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.59,8.41,5.48


## Step 2: Data Cleaning and Exploratory Data Analysis

In [250]:
df = unclean_df[['U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'OUTAGE.DURATION', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'PC.REALGSP.REL', 'POPPCT_URBAN']]
display(df.head())

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1,Minnesota,MRO,East North Central,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,3060,35.55,32.23,32.2,1.08,73.27
2,Minnesota,MRO,East North Central,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,1,30.03,34.21,35.73,1.09,73.27
3,Minnesota,MRO,East North Central,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,3000,28.1,34.5,37.37,1.07,73.27
4,Minnesota,MRO,East North Central,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,2550,31.99,33.54,34.44,1.07,73.27
5,Minnesota,MRO,East North Central,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,1740,33.98,36.21,29.78,1.09,73.27


In [251]:
df['OUTAGE.DURATION'] = df['OUTAGE.DURATION'].apply(lambda x: np.nan if x == 0 else x)
df = df[df['OUTAGE.DURATION'].notna()]

In [252]:
unclean_df[unclean_df['OUTAGE.DURATION'].isna()].head(10)

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,RES.SALES,COM.SALES,IND.SALES,TOTAL.SALES,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUSTOMERS,COM.CUSTOMERS,IND.CUSTOMERS,TOTAL.CUSTOMERS,RES.CUST.PCT,COM.CUST.PCT,IND.CUST.PCT,PC.REALGSP.STATE,PC.REALGSP.USA,PC.REALGSP.REL,PC.REALGSP.CHANGE,UTIL.REALGSP,TOTAL.REALGSP,UTIL.CONTRI,PI.UTIL.OFUSA,POPULATION,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
23,2015.0,7.0,Tennessee,TN,SERC,Central,1.2,warm,2015-07-30 00:00:00,13:00:00,NaN,NaN,intentional attack,vandalism,NaN,NaN,0,0.0,10.31,10.27,7.31,9.7,4285049,3322562,1891937,9499547,45.11,34.98,19.92,2.78e+06,4.76e+05,1245.0,3.26e+06,85.35,14.61,0.04,42457,49844,0.85,1.2,1692,282752,0.6,0.4,6.60e+06,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16
37,2016.0,7.0,Tennessee,TN,SERC,Central,-0.3,normal,2016-07-13 00:00:00,15:00:00,NaN,NaN,system operability disruption,public appeal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.81e+06,4.80e+05,1193.0,3.29e+06,85.39,14.58,0.04,43720,50660,0.86,3,1813,290712,0.62,0.5,6.65e+06,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16
48,2016.0,4.0,Tennessee,TN,SERC,Central,1.1,warm,2016-04-27 00:00:00,13:36:00,NaN,NaN,intentional attack,vandalism,NaN,NaN,0,0.0,10.22,9.64,5.39,8.8,2517024,2577433,1681946,6776403,37.14,38.04,24.82,2.81e+06,4.80e+05,1193.0,3.29e+06,85.39,14.58,0.04,43720,50660,0.86,3,1813,290712,0.62,0.5,6.65e+06,66.39,12.02,1450.3,1076.2,55.6,7.05,1.72,97.84,2.16,2.16
50,2014.0,6.0,Wisconsin,WI,MRO,East North Central,0,normal,2014-06-27 00:00:00,13:21:00,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,14.38,11.31,7.81,10.98,1706907,2012702,2037837,5757447,29.65,34.96,35.39,2.63e+06,3.46e+05,5465.0,2.98e+06,88.22,11.6,0.18,46676,49091,0.95,1.9,4680,268742,1.74,1.9,5.76e+06,70.15,14.35,2123.3,1671.5,32.5,3.47,0.9,82.69,17.31,3.05
183,2007.0,9.0,Texas,TX,WECC,South,-0.9,cold,2007-09-06 00:00:00,20:00:00,NaN,NaN,fuel supply emergency,NaN,NaN,NaN,NaN,NaN,12.44,9.88,7.76,10.3,13316662,10376502,9494213,33193019,40.12,31.26,28.6,9.17e+06,1.41e+06,168586.0,1.07e+07,85.28,13.15,1.57,49068,49126,1.0,2.5,27695,1169399,2.37,10.9,2.38e+07,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09
193,2014.0,4.0,Texas,TX,TRE,South,-0.2,normal,2014-04-03 00:00:00,00:00:00,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,12.06,8.21,5.86,8.55,8199586,10173132,9461319,27846934,29.45,36.53,33.98,1.01e+07,1.43e+06,101641.0,1.17e+07,86.86,12.27,0.87,52742,49091,1.07,2,28121,1421759,1.98,10.9,2.70e+07,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09
233,2014.0,6.0,Texas,TX,TRE,South,0,normal,2014-06-06 00:00:00,13:00:00,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,12.17,8.26,6.34,9.2,12861137,12571521,9260108,34707247,37.06,36.22,26.68,1.01e+07,1.43e+06,101641.0,1.17e+07,86.86,12.27,0.87,52742,49091,1.07,2,28121,1421759,1.98,10.9,2.70e+07,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09
240,2000.0,NaN,Texas,TX,FRCC,South,NaN,NaN,NaN,NaN,NaN,NaN,equipment failure,transformer outage,NaN,NaN,46,43000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.02e+06,1.09e+06,61280.0,9.30e+06,86.27,11.76,0.66,45102,44745,1.01,1.7,30908,944631,3.27,10.3,2.09e+07,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09
283,2015.0,12.0,Texas,TX,TRE,South,2.3,warm,2015-12-26 00:00:00,19:30:00,NaN,NaN,severe weather,NaN,NaN,NaN,NaN,70000.0,11.33,7.59,5.28,8.2,9873550,10401668,8456633,28745852,34.35,36.18,29.42,1.03e+07,1.43e+06,105936.0,1.18e+07,87.07,12.03,0.89,53707,49844,1.08,1.8,31256,1488049,2.1,11,2.75e+07,84.7,9.35,2435.3,1539.9,15.2,3.35,0.58,97.26,2.74,2.09
302,2000.0,8.0,Indiana,IN,ECAR,Central,-0.5,cold,2000-08-28 00:00:00,23:00:0

In [253]:
df['NERC.REGION'].isna().sum()
# there are no missing values in 'NERC.REGION'

np.int64(0)

In [254]:
df[df['CLIMATE.REGION'].isna()] # 5 rows with null values

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1516,Hawaii,HI,NaN,2008-12-26 00:00:00,18:13:00,2008-12-27 00:00:00,17:00:00,severe weather,1367.0,30.36,32.96,36.68,1.04,91.93
1517,Hawaii,PR,NaN,2011-05-02 00:00:00,17:06:00,2011-05-02 00:00:00,20:00:00,severe weather,174.0,29.26,34.29,36.45,1.03,91.93
1518,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-15 00:00:00,16:12:00,severe weather,543.0,29.75,33.33,36.92,1.03,91.93
1519,Hawaii,HECO,NaN,2006-06-01 00:00:00,14:12:00,2006-06-01 00:00:00,18:09:00,system operability disruption,237.0,29.78,33.48,36.75,1.03,91.93
1520,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-16 00:00:00,14:55:00,severe weather,1906.0,29.75,33.33,36.92,1.03,91.93


In [255]:
df[df['U.S._STATE'] == 'Hawaii']
# Hawaii does not have a 'CLIMATE.REGION'

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1516,Hawaii,HI,NaN,2008-12-26 00:00:00,18:13:00,2008-12-27 00:00:00,17:00:00,severe weather,1367.0,30.36,32.96,36.68,1.04,91.93
1517,Hawaii,PR,NaN,2011-05-02 00:00:00,17:06:00,2011-05-02 00:00:00,20:00:00,severe weather,174.0,29.26,34.29,36.45,1.03,91.93
1518,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-15 00:00:00,16:12:00,severe weather,543.0,29.75,33.33,36.92,1.03,91.93
1519,Hawaii,HECO,NaN,2006-06-01 00:00:00,14:12:00,2006-06-01 00:00:00,18:09:00,system operability disruption,237.0,29.78,33.48,36.75,1.03,91.93
1520,Hawaii,HECO,NaN,2006-10-15 00:00:00,07:09:00,2006-10-16 00:00:00,14:55:00,severe weather,1906.0,29.75,33.33,36.92,1.03,91.93


In [256]:
pd.set_option('display.max_rows', None)

In [257]:
df['CAUSE.CATEGORY'].isna().sum()
# all power outages have a valid cause

np.int64(0)

In [258]:
df[df['RES.PERCEN'].isna()]

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
104,Michigan,RFC,East North Central,2016-07-08 00:00:00,19:00:00,2016-07-09 00:00:00,00:00:00,severe weather,300.0,NaN,NaN,NaN,0.86,74.57
172,Texas,TRE,South,2016-07-05 00:00:00,02:45:00,2016-07-06 00:00:00,03:00:00,severe weather,1455.0,NaN,NaN,NaN,1.05,84.7
219,Texas,TRE,South,2016-07-09 00:00:00,17:45:00,2016-07-11 00:00:00,14:00:00,severe weather,2655.0,NaN,NaN,NaN,1.05,84.7
418,Washington,WECC,Northwest,2016-07-13 00:00:00,13:00:00,2016-07-13 00:00:00,13:01:00,intentional attack,1.0,NaN,NaN,NaN,1.14,84.05
606,Pennsylvania,RFC,Northeast,2016-07-23 00:00:00,15:15:00,2016-07-23 00:00:00,19:53:00,system operability disruption,278.0,NaN,NaN,NaN,1.01,78.66
744,North Carolina,SERC,Southeast,2016-07-08 00:00:00,20:50:00,2016-07-09 00:00:00,19:25:00,severe weather,1355.0,NaN,NaN,NaN,0.87,66.09
763,North Carolina,SERC,Southeast,2016-07-07 00:00:00,05:53:00,2016-07-07 00:00:00,08:40:00,intentional attack,167.0,NaN,NaN,NaN,0.87,66.09
828,Oregon,WECC,Northwest,2016-07-02 00:00:00,04:00:00,2016-07-04 00:00:00,00:40:00,intentional attack,2680.0,NaN,NaN,NaN,1.0,81.03
901,Delaware,RFC,Northeast,2016-07-21 00:00:00,06:18:00,2016-07-21 00:00:00,14:45:00,intentional attack,507.0,NaN,NaN,NaN,1.25,83.3
1420,Oklahoma,SPP,South,2016-07-14 00:00:00,14:44:00,2016-07-15 00:00:00,04:00:00,severe weather,796.0,NaN,NaN,NaN,0.88,66.24


In [259]:
df[df['COM.PERCEN'].isna()].equals(df[df['RES.PERCEN'].isna()])

True

In [260]:
df[df['IND.PERCEN'].isna()].equals(df[df['RES.PERCEN'].isna()])
# NaN values for percentages are across the same rows

True

In [261]:
# imputing 'PERCEN' columns using the 
df['RES.PERCEN'] = df['RES.PERCEN'].fillna(df['RES.PERCEN'].mean())
df['COM.PERCEN'] = df['COM.PERCEN'].fillna(df['COM.PERCEN'].mean())
df['IND.PERCEN'] = df['IND.PERCEN'].fillna(df['IND.PERCEN'].mean())
display(df.head(15))

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_451/1602715756.py:2: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_451/1602715756.py:3: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/jk/ll2fwx_55klc6t5gz7x_24440000gn/T/ipykernel_451/1602715756.py:4: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('f

,U.S._STATE,NERC.REGION,CLIMATE.REGION,OUTAGE.START.DATE,OUTAGE.START.TIME,OUTAGE.RESTORATION.DATE,OUTAGE.RESTORATION.TIME,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN
1,Minnesota,MRO,East North Central,2011-07-01 00:00:00,17:00:00,2011-07-03 00:00:00,20:00:00,severe weather,3060.0,35.55,32.23,32.20,1.08,73.27
2,Minnesota,MRO,East North Central,2014-05-11 00:00:00,18:38:00,2014-05-11 00:00:00,18:39:00,intentional attack,1.0,30.03,34.21,35.73,1.09,73.27
3,Minnesota,MRO,East North Central,2010-10-26 00:00:00,20:00:00,2010-10-28 00:00:00,22:00:00,severe weather,3000.0,28.10,34.50,37.37,1.07,73.27
4,Minnesota,MRO,East North Central,2012-06-19 00:00:00,04:30:00,2012-06-20 00:00:00,23:00:00,severe weather,2550.0,31.99,33.54,34.44,1.07,73.27
5,Minnesota,MRO,East North Central,2015-07-18 00:00:00,02:00:00,2015-07-19 00:00:00,07:00:00,severe weather,1740.0,33.98,36.21,29.78,1.09,73.27
6,Minnesota,MRO,East North Central,2010-11-13 00:00:00,15:00:00,2010-11-14 00:00:00,22:00:00,severe weather,1860.0,31.19,33.24,35.54,1.07,73.27
7,Minnesota,MRO,East North Central,2010-07-17 00:00:00,20:30:00,2010-07-19 00:00:00,22:00:00,severe weather,2970.0,34.31,32.95,32.70,1.07,73.27
8,Minnesota,MRO,East North Central,2005-06-08 00:00:00,04:00:00,2005-06-10 00:00:00,22:00:00,severe weather,3960.0,33.59,34.52,31.86,1.09,73.27
9,Minnesota,MRO,East North Central,2015-03-16 00:00:00,07:31:00,2015-03-16 00:00:00,10:06:00,intentional attack,155.0,32.94,34.95,32.07,1.09,73.27
10,Minnesota,MRO,East North Central,2013-06-21 00:00:00,17:39:00,2013-06-24 00:00:00,06:00:00,severe weather,3621.0,30.75,34.52,34.70,1.08,73.27


In [262]:
df['PC.REALGSP.REL'].isna().sum()
# no null values

np.int64(0)

In [263]:
df['POPPCT_URBAN'].isna().sum()
# no null values again

np.int64(0)

In [264]:
# converting to pd.Timestamp
outage_start_date_str = pd.to_datetime(df['OUTAGE.START.DATE']).astype(str)
outage_start_time_str = pd.to_datetime(df['OUTAGE.START.TIME'], format='%H:%M:%S').dt.time.astype(str)
outage_start = pd.to_datetime(outage_start_date_str + ' ' + outage_start_time_str)

outage_restoration_date_str = pd.to_datetime(df['OUTAGE.RESTORATION.DATE']).astype(str)
outage_restoration_time_str = pd.to_datetime(df['OUTAGE.RESTORATION.TIME'], format='%H:%M:%S').dt.time.astype(str)
outage_restoration = pd.to_datetime(outage_restoration_date_str + ' ' + outage_restoration_time_str)

df['OUTAGE.START'] = outage_start
df['OUTAGE.RESTORATION'] = outage_restoration
df = df.drop(columns=['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'])

In [402]:
display(df.head()) # head of the cleaned dataframe

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1,Minnesota,MRO,East North Central,severe weather,3060.0,35.55,32.23,32.20,1.08,73.27,2011-07-01 17:00:00,2011-07-03 20:00:00
2,Minnesota,MRO,East North Central,intentional attack,1.0,30.03,34.21,35.73,1.09,73.27,2014-05-11 18:38:00,2014-05-11 18:39:00
3,Minnesota,MRO,East North Central,severe weather,3000.0,28.10,34.50,37.37,1.07,73.27,2010-10-26 20:00:00,2010-10-28 22:00:00
4,Minnesota,MRO,East North Central,severe weather,2550.0,31.99,33.54,34.44,1.07,73.27,2012-06-19 04:30:00,2012-06-20 23:00:00
5,Minnesota,MRO,East North Central,severe weather,1740.0,33.98,36.21,29.78,1.09,73.27,2015-07-18 02:00:00,2015-07-19 07:00:00


In [266]:
# I don't know what to do for no climate region for Hawaii and the NaN PERCEN values.

### Univariate Analysis

In [308]:
cause_counts = pd.DataFrame(df['CAUSE.CATEGORY'].value_counts().reset_index())
px.pie(cause_counts, values='count', names='CAUSE.CATEGORY')
# This plot shows that the most common cause of power outages is severe weather, and the least common is fuel supply emergency


In [380]:
px.histogram(df, x='U.S._STATE')

### Bivariate Analysis

In [382]:
px.bar(df.groupby('CLIMATE.REGION')['OUTAGE.DURATION'].mean().sort_values(ascending=False), y='OUTAGE.DURATION')
# this bar plot shows that the 'CLIMATE.REGION' in which power takes the longest to restore on average is 'East North Central'. The minimum is at 'West North Central'.

In [383]:
px.bar(df.groupby('CLIMATE.REGION')['OUTAGE.DURATION'].count().sort_values(ascending=False), y='OUTAGE.DURATION')
# this bar plot shows that the 'CLIMATE.REGION' in which power outages happen the most frequently is different, becoming 'Northeast' this time. Both have the minimum at 'West North Central'.

### Interesting Aggregates

In [408]:
# Funciton to calculate the average time of day of a pd.Timestamp series, without including the date
def avg_time(series):
    mean_seconds = (series.dt.hour * 3600 + series.dt.minute * 60 + series.dt.second).mean()
    return pd.to_datetime(mean_seconds, unit='s').time()

In [409]:
df.groupby('U.S._STATE')['OUTAGE.START'].agg(avg_time)
pd.pivot_table(df, values='OUTAGE.START', index='CAUSE.CATEGORY', columns=['CLIMATE.REGION'], aggfunc=avg_time)

CLIMATE.REGION,Central,East North Central,Northeast,Northwest,South,Southeast,Southwest,West,West North Central
CAUSE.CATEGORY,,,,,,,,,
equipment failure,13:00:48,08:35:00,11:15:45,16:47:00,10:46:00,16:50:15,15:39:36,12:00:20,09:46:00
fuel supply emergency,08:36:15,07:50:15,12:04:08.571428,01:48:00,09:52:30,NaN,09:14:00,13:17:18,NaN
intentional attack,11:23:35,12:19:22.105263,10:26:53.195876,12:21:12.923076,10:55:33.333333,12:05:40,11:15:06.101694,10:32:22,12:51:00
islanding,18:19:00,12:01:00,22:44:00,16:03:20,18:13:30,NaN,09:22:00,12:26:42.857142,14:50:12
public appeal,14:00:00,14:07:00,13:00:00,14:02:00,13:36:15.714285,15:00:00,10:00:00,14:08:06.666666,08:10:30
severe weather,13:11:27.022900,13:32:09.230769,13:30:53.142857,09:33:52.800000,12:26:10.754716,13:36:49.043478,09:15:06,11:46:49.253731,17:20:00
system operability disruption,15:16:30,11:54:20,15:21:38.571428,15:28:00,13:27:18.461538,12:11:44,14:02:45,12:54:09.230769,NaN


## Step 3: Assessment of Missingness

In [412]:
df.head()

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1,Minnesota,MRO,East North Central,severe weather,3060.0,35.55,32.23,32.20,1.08,73.27,2011-07-01 17:00:00,2011-07-03 20:00:00
2,Minnesota,MRO,East North Central,intentional attack,1.0,30.03,34.21,35.73,1.09,73.27,2014-05-11 18:38:00,2014-05-11 18:39:00
3,Minnesota,MRO,East North Central,severe weather,3000.0,28.10,34.50,37.37,1.07,73.27,2010-10-26 20:00:00,2010-10-28 22:00:00
4,Minnesota,MRO,East North Central,severe weather,2550.0,31.99,33.54,34.44,1.07,73.27,2012-06-19 04:30:00,2012-06-20 23:00:00
5,Minnesota,MRO,East North Central,severe weather,1740.0,33.98,36.21,29.78,1.09,73.27,2015-07-18 02:00:00,2015-07-19 07:00:00


In [415]:
df.isnull().any()

U.S._STATE            False
NERC.REGION           False
CLIMATE.REGION         True
CAUSE.CATEGORY        False
OUTAGE.DURATION       False
RES.PERCEN            False
COM.PERCEN            False
IND.PERCEN            False
PC.REALGSP.REL        False
POPPCT_URBAN          False
OUTAGE.START          False
OUTAGE.RESTORATION    False
dtype: bool

In [416]:
df[df['CLIMATE.REGION'].isna()]

,U.S._STATE,NERC.REGION,CLIMATE.REGION,CAUSE.CATEGORY,OUTAGE.DURATION,RES.PERCEN,COM.PERCEN,IND.PERCEN,PC.REALGSP.REL,POPPCT_URBAN,OUTAGE.START,OUTAGE.RESTORATION
1516,Hawaii,HI,NaN,severe weather,1367.0,30.36,32.96,36.68,1.04,91.93,2008-12-26 18:13:00,2008-12-27 17:00:00
1517,Hawaii,PR,NaN,severe weather,174.0,29.26,34.29,36.45,1.03,91.93,2011-05-02 17:06:00,2011-05-02 20:00:00
1518,Hawaii,HECO,NaN,severe weather,543.0,29.75,33.33,36.92,1.03,91.93,2006-10-15 07:09:00,2006-10-15 16:12:00
1519,Hawaii,HECO,NaN,system operability disruption,237.0,29.78,33.48,36.75,1.03,91.93,2006-06-01 14:12:00,2006-06-01 18:09:00
1520,Hawaii,HECO,NaN,severe weather,1906.0,29.75,33.33,36.92,1.03,91.93,2006-10-15 07:09:00,2006-10-16 14:55:00


In our cleaned dataset, the only column with missing values is `'CLIMATE.REGION'`. Since the only rows for which values are missing are for `'U.S._STATE' == Hawaii`, this missingness depends on the `'U.S._STATE'` column, making it MAR. Before imputing missing values in `'RES.PERCEN'`, `'COM.PERCEN'`, and `'IND.PERCEN'`, the missing values did not seem to depend on themselves, and hence we cannot be sure if the missingness mechanism is NMAR.

In [425]:
df.groupby('NERC.REGION')['CLIMATE.REGION']

AttributeError: 'SeriesGroupBy' object has no attribute 'mode'

## Step 4: Hypothesis Testing

In [ ]:
# TODO

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO